In [1]:
# A python version of the blockchain constructed using JavaScript
# on the video series "Building a blockchain with JavaScript".
# Playlist link: https://www.youtube.com/watch?v=zVqczFZr124&list=PLzvRQMJ9HDiTqZmbtFisdXFxul5k0F-Q4&index=1
import numpy as np
import hashlib
from datetime import datetime
from Crypto.PublicKey import ECC

In [2]:
def encrypt_string(hash_string: str) -> str:
    """ Returns a string encrypted using SHA256.
    
    Args:
        hash_string (string): Information that will be encrypted using SHA256.
    Returns:
        sha_signature (string): Encrypted SHA256 version of the hash_string.
    """
    sha_signature = hashlib.sha256(hash_string.encode()).hexdigest()
    return sha_signature

In [3]:
class Transaction:
    def __init__(self, senderAddress: str, receiverAddress: str, amount: float):
        """ Initialization of the Transaction class, whose is a definition of the action of
        send or receive some amount of a cryptocurrency.
        
        Properties:
            senderAddress (string): Wallet address of the sender in the transaction.
            receiverAddress (string): Wallet addres of the receiver in the transaction.
            amount (float): The quantity of the cryptocurrency that are going to be transferred.
        """
        self.senderAddress = senderAddress
        self.receiverAddress = receiverAddress
        self.amount = amount
        
    #def calculateHash(self):
    #    """
    #    """
    #    return encrypt_string(self.senderAddress + self.receiverAddress + str(self.amount))
    
    #def signTransaction(signingKey):
    #    transactionHash = self.calculateHash()
    #    signature = ECC.generate(curve='secp256r1') 

In [4]:
class Block:
    def __init__(self, timestamp: datetime, transactions: list, previous_hash = ''):
        """ Initialization of the Block class.
        
        Properties:
            timestamp (datetime): The date of the creation of the block.
            transactions (list): A list with the transactions that was processed
            in this block.
            previous_hash (string): The hash of the previous block that guarantees
            the link between the blocks in the blockchain.
            nonce (integer): A random auxiliary variable for the process of a
            generation of a hash.
            hash (string): A encrypted string generated using the others properties
            of the block through the use of SHA256 encryption method.
        """
        self.timestamp = timestamp
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = np.random.randint(0, 32768)
        self.hash = self.calculateHash()
        
    def calculateHash(self) -> str:
        """ A function that determines the hash of a block given its properties.
        
        Returns:
            block_hash (string): A encrypted string using SHA256.
        """
        block_hash = encrypt_string(str(self.timestamp) + str(self.transactions) + self.previous_hash + str(self.nonce))
        return block_hash
    
    def mineBlock(self, difficulty: int) -> None:
        """ Function that defines the process of mining a block.
        
        Args:
            difficulty (int): Number of zeros that the hash must have
            in the first n=difficulty chars to be considered valid.
        """
        zeros_string = ''
        for i in range(difficulty):
            zeros_string = zeros_string + '0'
        while self.hash[0:difficulty] != zeros_string:
            self.nonce += 1
            self.hash = self.calculateHash()   

In [5]:
class Blockchain:
    def __init__(self):
        """ Initialization of the Blockchain class.
        
        Properties:
            chain (list): A list of objects of Block type.
            difficulty (integer): Set the quantity of zeros that the hash need to has for be
            considered a valid hash. That number increase the difficulty to obtain a hash with
            that form.
            pendingTransactions (list): A list with the pending transactions in the chain.
            miningReward (float): The amount of cryptocurrency that the miner will receive
            after winning the mining process.
        """
        self.chain = [self.createGenesisBlock()]
        self.difficulty = 2
        self.pendingTransactions = []
        self.miningReward = 100
        
    def createGenesisBlock(self) -> Block:
        """ Function that beget the genesis block of the blockchain.
        
        Returns:
            Block: The genesis block.
        """
        return Block(datetime.now(), "Genesis block", "0")
    
    def getLatestBlock(self) -> Block:
        """ A function that gives us the last block in the blockchain.
        
        Returns:
            Block: The last object of Block type in the blockchain.
        """
        return self.chain[len(self.chain)-1]
    
    def minePendingTransactions(self, miningRewardAddress: str) -> None:
        """ Adds the pending transactions in the transactions of the mined block.
        
        Args:
            miningRewardAddress (string): The address of the wallet of the winner of
            mining process.
        """
        block = Block(datetime.now(), self.pendingTransactions, self.getLatestBlock().hash)
        block.mineBlock(self.difficulty)
        self.chain.append(block)
        self.pendingTransactions = [Transaction(None, miningRewardAddress, self.miningReward)]
        
    def addAPendingTransaction(self, transaction: Transaction) -> None:
        """ Insert an object of Transaction type in the last position of a
        list of pending transactions.
        
        Args:
            transaction (Transaction): The transaction that is going to be added
            to the list.
        
        """
        self.pendingTransactions.append(transaction)
        
    def getBalanceOfAWallet(self, address: str) -> float:
        """ Given an address, returns the balance of the wallet with this address.
        
        Args:
            address (string):
            
        Returns:
            balance (float): The amount of cryptocurrency stored in
            the wallet with the given address.
        """
        balance = 0
        for i in range(1, len(self.chain)):
            for j in range(len(self.chain[i].transactions)):
                if self.chain[i].transactions[j].senderAddress == address:
                    balance -= self.chain[i].transactions[j].amount
                elif self.chain[i].transactions[j].receiverAddress == address:
                    balance += self.chain[i].transactions[j].amount
                    
        return balance
        
    def isValid(self) -> bool:
        """ A function that checks if the blockchain is valid or not.
        The blockchain will be considered invalid when the hash of the
        current block, that is under analysis, not match the hash obtained
        through calculateHash() method. A blockchain also can be considered
        invalid when the property previous_hash of the current_block is different
        from the hash of the previous_block.
        
        Returns:
            bool (True or False)
        """
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]
            
            if current_block.hash != current_block.calculateHash():
                return False
            if current_block.previous_hash != previous_block.hash:
                return False
            
        return True
    
    #def addBlock(self, block: Block) -> None:
    #    """ Adds a block to the Blockchain. 
    #    
    #    Args:
    #        block (Block): An object of Block type that contains some transactions.
    #    """
    #    block.previous_hash = self.getLatestBlock().hash
    #    block.mineBlock(self.difficulty)
    #    self.chain.append(block)

In [6]:
# Creating the object blockchain.
blockchain = Blockchain()

In [7]:
# Printing the hash of the genesis block.
print(blockchain.chain[0].hash)

420aaa6621f47188598bfef6470209490a631e39b3feeade9b7e87ff504d1c3e


In [8]:
# Creating transactions and adding them in the pending transactions of the blockchain.
blockchain.addAPendingTransaction(Transaction('public_key1', 'public_key2', 100))
blockchain.addAPendingTransaction(Transaction('public_key2', 'public_key1', 50))

In [9]:
# Processing the pending transactions.
blockchain.minePendingTransactions('public_key3')

In [10]:
# Checking the balance of the wallet of the miner.
# Observation: Here the balance is equal to zero,
# because the mining reward transaction is in the pending transactions.
print('The balance of the wallet that has public_key3 as address is equal to: {}'.format(blockchain.getBalanceOfAWallet('public_key3')))

The balance of the wallet that has public_key3 as address is equal to: 0


In [11]:
# Processing the pending transactions.
blockchain.minePendingTransactions('public_key3')

In [12]:
# Checking the balance of the wallet of the miner.
print('The balance of the wallet that has public_key3 as address is equal to: {}'.format(blockchain.getBalanceOfAWallet('public_key3')))

The balance of the wallet that has public_key3 as address is equal to: 100


In [13]:
blockchain.addAPendingTransaction(Transaction('public_key5', 'public_key4', 25))
blockchain.addAPendingTransaction(Transaction('public_key4', 'public_key6', 78))

In [14]:
blockchain.minePendingTransactions('public_key3')

In [15]:
print('The balance of the wallet that has public_key3 as address is equal to: {}'.format(blockchain.getBalanceOfAWallet('public_key3')))

The balance of the wallet that has public_key3 as address is equal to: 200


In [16]:
blockchain.isValid()

True

In [8]:
blockchain.addBlock(bloco)

In [9]:
last_block = blockchain.getLatestBlock()

In [10]:
print(last_block.hash)

00409c381398a186d2f08f1bcdab99ef342cc1643f446b048dbf1b051421340f


In [11]:
blockchain.isChainValid()

True

In [12]:
blockchain.chain.append(Block(2, datetime.now(), "Invalid block inserted", encrypt_string(str(np.random.randint(0,2048)))))

In [13]:
blockchain.isChainValid()

False

In [56]:
key = ECC.generate(curve='secp256r1')

In [57]:
print(key.export_key(format='DER', passphrase='veris', use_pkcs8=True, protection='PBKDF2WithHMAC-SHA1AndAES128-CBC'))

b"0\x81\xde0I\x06\t*\x86H\x86\xf7\r\x01\x05\r0<0\x1b\x06\t*\x86H\x86\xf7\r\x01\x05\x0c0\x0e\x04\x08\xe3\x86\x194\xc7\xe35\xb2\x02\x02\x03\xe80\x1d\x06\t`\x86H\x01e\x03\x04\x01\x02\x04\x10\xdb\x13\xaf\xfd\x8e\xcd\x0e\xc6\x00\xab\x1e\xe6\x82Z\xa3S\x04\x81\x90m\xbe\x96>W\x8f5\xa1\x9f\x07%-R\x9c/\x800\xfch_]\xa0\x02\xf5\x8a\x16>\xa5\x8d\x8a0\x12^\xa2\xb6W\xa9f\x04\xffx\xff\x18]\xc3'\xdaTVfD\x88I\x1d\xb9\xa2\xd4B\x14c\x91;\x8f\xdb\x95\x86\xf0\xa7\x1bm\xbe(\x1b\xd5^\x838\xf6'\x83\rk\xe4m,\xd7|\xa9\x1c\xc3\xeeh\xc1|B\x1fw\xacZA\xa9xR\x03'\x1dx\xb65K;\x02\xbc\xde\xf1W,\xfaO\xfe\x02g\x91\xbfg_/;\x94M\xcct\xa4\x87sx>\xfe\xdc{Y\x9d\xcf\xdc"


In [58]:
print(hex(int.from_bytes(key.export_key(format='DER'), "big", signed=True)))

0x308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b0201010420d9371d018d6ec179098cd8ee84bae34ba52afd5603ebb7f50a921296f8f5a4c8a14403420004cf92950b317be762d3229f6944e0a9d7b8286de7479b2781ed6f8cc0999ac1955ec303146bfe149d5172abd1e72fe68d78a78d951265463c6e724c031f3d0335


In [59]:
print(key.public_key().export_key(format='DER'))

b'0Y0\x13\x06\x07*\x86H\xce=\x02\x01\x06\x08*\x86H\xce=\x03\x01\x07\x03B\x00\x04\xcf\x92\x95\x0b1{\xe7b\xd3"\x9fiD\xe0\xa9\xd7\xb8(m\xe7G\x9b\'\x81\xedo\x8c\xc0\x99\x9a\xc1\x95^\xc3\x03\x14k\xfe\x14\x9dQr\xab\xd1\xe7/\xe6\x8dx\xa7\x8d\x95\x12eF<nrL\x03\x1f=\x035'


In [60]:
print(hex(int.from_bytes(key.public_key().export_key(format='DER'), "big", signed=True)))

0x3059301306072a8648ce3d020106082a8648ce3d03010703420004cf92950b317be762d3229f6944e0a9d7b8286de7479b2781ed6f8cc0999ac1955ec303146bfe149d5172abd1e72fe68d78a78d951265463c6e724c031f3d0335
